<a href="https://colab.research.google.com/github/DDDCR/NLP/blob/NER/Entities_extraction_Resume_(CV).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install spacy-transformers
!pip install -U spacy

In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [ ]:
spacy.__version__

In [ ]:
!nvidia-smi

In [5]:
!python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

2024-01-10 18:06:34.954076: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 18:06:34.954152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 18:06:34.955388: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 18:06:36.011712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


###Preparing Data


In [ ]:
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json'))

In [ ]:
#Preparing Training Data
def get_spacy_doc(file, data): # file is for error data
  nlp = spacy.blank('en')
  db = DocBin()

  for text,annotations in tqdm(data):
    doc = nlp.make_doc(text)
    annotations = annotations['entities']

    ents = []
    entity_indices = []

    for start, end, label in annotations:
      skip_entity = False  #check if the entities are overlap:
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity == True:
        continue
      entity_indices = entity_indices + list(range(start, end)) #update entity_indices

      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        err_data = str([start, end]) + '    ' + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [ ]:
#Spliting data
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3) #0.7 as training data, 0.3 as test data

In [ ]:
file = open('error.txt','w')

db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

file.close()

###Training

In [7]:
!python -m spacy train /content/config.cfg --output ./output --paths.train /content/relations_training.spacy --paths.dev /content/relations_dev.spacy --gpu-id 0

2024-01-10 18:08:21.053796: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 18:08:21.053845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 18:08:21.055088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 18:08:22.124276: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly init

### Model Test


In [8]:
nlp = spacy.load('/content/output/model-best')

In [12]:
doc = nlp('i have 2+ years of non-internship professional software development experience Programming experience with at least one modern language such as Java, C++, or C# including object-oriented design. 1+ years of experience contributing to the architecture and design (architecture, design patterns, reliability and scaling) of new and current systems. Bachelor / MS Degree in Computer Science. Preferably a PhD in data science. 8+ years of professional experience in software development. 2+ years of experience in project management. Experience in mentoring junior software engineers to improve their skills, and make them more effective, product software engineers. Experience in data structures, algorithm design, complexity analysis, object-oriented design. 3+ years experience in at least one modern programming language such as Java, Scala, Python, C++, C# Experience in professional software engineering practices & best practices for the full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations Experience in communicating with users, other technical teams, and management to collect requirements, describe software product features, and technical designs. Experience with building complex software systems that have been successfully delivered to customers Proven ability to take a project from scoping requirements through actual launch of the project, with experience in the subsequent operation of the system in production')
for ent in doc.ents:
  print(ent.text," --> ", ent.label_)

2+ years  -->  EXPERIENCE
non-internship  -->  SKILLS
Programming  -->  SKILLS
Java  -->  SKILLS
C++  -->  SKILLS
C#  -->  SKILLS
object-oriented design  -->  SKILLS
1+ years  -->  EXPERIENCE
Bachelor / MS  -->  DIPLOMA
Computer Science  -->  DIPLOMA_MAJOR
PhD  -->  DIPLOMA
data science  -->  DIPLOMA_MAJOR
8+ years  -->  EXPERIENCE
professional experience  -->  SKILLS
software development  -->  SKILLS
2+ years  -->  EXPERIENCE
project management  -->  SKILLS
3+ years  -->  EXPERIENCE
programming language  -->  SKILLS
Java  -->  SKILLS
Scala  -->  SKILLS
Python  -->  SKILLS
C++  -->  SKILLS
C#  -->  SKILLS
